In [1]:
from vosk import Model, KaldiRecognizer, SetLogLevel
import sys
import os
import wave
import subprocess
import srt
import json
import datetime
import pyaudio
import time
import ctypes
from multiprocessing import Process, Value, Array

#manager = multiprocessing.Manager()

SetLogLevel(-1)
model_path = "../../asr/vosk/model"

if not os.path.exists(model_path):
	print (f"Please download the model from https://alphacephei.com/vosk/models and unpack as 'model' in {model_path}")
	exit (1)

sample_rate=16000
model = Model(model_path)
rec = KaldiRecognizer(model, sample_rate)
rec.SetWords(True)

# typecodes : https://docs.python.org/3/library/array.html#module-array
smile_detected = Value(ctypes.c_bool, False, lock=False)

# flow :
# asr() -> locate() -> smile_detection() -> take_pic()

def take_pic():
	print("taking a picture ...")
	time.sleep(2)
	print("took a picture !")

# Sound + Opencv based ?
def locate():
	print("moving ...")
	time.sleep(3)
	print("done moving !")
	pass

# Opencv loop
def smile_detection():
	time.sleep(5)
	while True:
			for i in range(10):
				time.sleep(5)
				if i%5 == 0:
					#manager.Value("smile_detected", True)
					print("smile detected !")
					smile_detected.value = True
					#take_pic()
					#manager.Value("smile_detected", False)
					#break
				else:
					print(":(")
					smile_detected.value = False
				time.sleep(1)
			time.sleep(0.5)

# ASR loop
def asr():
	p = pyaudio.PyAudio()
	stream = p.open(format=pyaudio.paInt16, channels=1, rate=16000, input=True, frames_per_buffer=8000)
	stream.start_stream()

	while True:
		data = stream.read(4000)
		if len(data) == 0:
			break
		if rec.AcceptWaveform(data):
			result = rec.FinalResult()
			#print(result)
			jres = json.loads(result)
			if "result" in jres:
				sentence = jres["text"]
				if "photo" in sentence:
					locate()
					while True:
						if smile_detected.value:
							print("smile detected, time to take a pic !")
							take_pic()
							break

try:
	smile_detection_p = Process(target=smile_detection)
	asr_p = Process(target=asr)

	smile_detection_p.start()
	asr_p.start()

	smile_detection_p.join()
	asr_p.join()

except (KeyboardInterrupt, SystemExit):
	print("Exiting...")
	smile_detection_p.terminate()
	asr_p.terminate()

	smile_detection_p.join()
	asr_p.join()

ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5233:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2660:(snd_pcm_open_noupdate) Unknown PCM sysdefault
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4745:(_snd_config_evaluate) function snd_func_concat returned error

smile detected !
:(
moving ...
done moving !
:(
:(
:(
smile detected !
smile detected, time to take a pic !
taking a picture ...
took a picture !
:(
:(
moving ...
done moving !
:(
:(
smile detected !
smile detected, time to take a pic !
taking a picture ...
took a picture !
:(
:(
Exiting...
